In [0]:
%pylab inline

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [0]:
from dataikuapi.dss.recipe import PredictionScoringRecipeCreator
client = dataiku.api_client()
project = client.get_project("DKU_CHURN")
builder = PredictionScoringRecipeCreator("my_scoring_recipe", project)
builder.with_input_model("5HExUjQ1")
builder.with_input("unlabeled_customers_prepared")
builder.with_new_output("my_output_dataset", "jordan-s3-bucket")

# Or for a filesystem output connection
# builder.with_new_output("my_output_dataset, "filesystem_managed", format_option_id="CSV_EXCEL_GZIP")

new_recipe = builder.build()

In [0]:
print(new_recipe)

In [0]:
import dataiku
client = dataiku.api_client()
project = client.get_project("DKU_CHURN")
sm = project.list_saved_models()
print(sm)

In [0]:
p = client.get_project("DKU_CHURN")

# Create a new ML Task to predict the variable "target" from "trainset"
mltask = p.create_prediction_ml_task(
    input_dataset="unlabeled_customers_prepared",
    target_variable="my_output_dataset",
    ml_backend_type='py37', # ML backend to use
    guess_policy='DEFAULT' # Template to use for setting default parameters
)

# Wait for the ML task to be ready
mltask.wait_guess_complete()

# Obtain settings, enable GBT, save settings
settings = mltask.get_settings()
settings.set_algorithm_enabled("RANDOM_FOREST_REGRESSION", True)
settings.save()